# FiftyOne + Twelve Labs   + Mosaic AI Vector Search with Databricks  <img src="assets/voxel51_logo.png" alt="Image2" width="40"/><img src="assets/twelve_labs_logo.jpg" alt="Image2" width="40"/><img src="assets/db_logo.png" alt="Image1" width="80"/>
This notebook demonstrates how to build a complete visual search workflow using **FiftyOne**, **Twelve Labs Embeddings** and **Mosaic AI Vector Search on Databricks**.

You will learn how to:
- Set up your Databricks catalog and vector search endpoint
- Load and index embeddings using Twelve Labs + FiftyOne
- Query by image and text
- Visualize results in the FiftyOne App

🧠 This integration helps you scale visual search over large datasets with a cloud-native vector database.

👉 For more, see the official [FiftyOne + Mosaic AI docs](https://docs.voxel51.com/integrations/mosaic.html) or the [Twelve Labs FiftyOne Plugin](https://github.com/danielgural/semantic_video_search)


<img src="assets/fo_tl_db_diagram.png" alt="Image2" width="600"/>

In [ ]:
# Install necessary packages
!pip install fiftyone databricks-vectorsearch  python-dotenv  umap-learn twelvelabs

In [ ]:
!fiftyone plugins download https://github.com/danielgural/semantic_video_search

## 🔐 Set Up Environment Variables
You need access to a Databricks account with **Vector Search enabled**.
Follow these steps:
1. **Create a Catalog**: Go to `Catalog` → `Add Data` → `Create a new Catalog`
2. **Create a Vector Search Endpoint**: Go to `Compute` → `Vector Search` → `Create`
3. **Create a Schema** inside your Catalog (FiftyOne will handle the columns later)

⚠️ You must have a Personal Access Token for authentication.

### 🔧 Option 1: Use a `.env` File
You can store your credentials securely in a `.env` file:
```bash
FIFTYONE_BRAIN_SIMILARITY_MOSAIC_WORKSPACE_URL=https://your.cloud.databricks.com/
FIFTYONE_BRAIN_SIMILARITY_MOSAIC_PERSONAL_ACCESS_TOKEN=your_token
FIFTYONE_BRAIN_SIMILARITY_MOSAIC_CATALOG_NAME=your_catalog
FIFTYONE_BRAIN_SIMILARITY_MOSAIC_SCHEMA_NAME=your_schema
FIFTYONE_BRAIN_SIMILARITY_MOSAIC_ENDPOINT_NAME=your_endpoint
TL_API_KEY=your_Twelve_Labs_API_key
```

In [ ]:
#from dotenv import load_dotenv
#load_dotenv()

### 🔧 Option 2: Set Environment Variables in Code
This is useful for notebooks or ephemeral environments:
```python
import os
os.environ["FIFTYONE_BRAIN_SIMILARITY_MOSAIC_WORKSPACE_URL"] = "https://your.cloud.databricks.com/"
os.environ["FIFTYONE_BRAIN_SIMILARITY_MOSAIC_PERSONAL_ACCESS_TOKEN"] = "your_token"
os.environ["FIFTYONE_BRAIN_SIMILARITY_MOSAIC_CATALOG_NAME"] = "your_catalog"
os.environ["FIFTYONE_BRAIN_SIMILARITY_MOSAIC_SCHEMA_NAME"] = "your_schema"
os.environ["FIFTYONE_BRAIN_SIMILARITY_MOSAIC_ENDPOINT_NAME"] = "your_endpoint"
os.environ["DATABRICKS_HOST"] = os.environ["FIFTYONE_BRAIN_SIMILARITY_MOSAIC_WORKSPACE_URL"]
os.environ["DATABRICKS_TOKEN"] = os.environ["FIFTYONE_BRAIN_SIMILARITY_MOSAIC_PERSONAL_ACCESS_TOKEN"]
os.environ["TL_API_KEY"]=your_Twelve_Labs_API_key
```

In [ ]:
# import os
# os.environ["FIFTYONE_BRAIN_SIMILARITY_MOSAIC_WORKSPACE_URL"] = "https://your.cloud.databricks.com/"
# os.environ["FIFTYONE_BRAIN_SIMILARITY_MOSAIC_PERSONAL_ACCESS_TOKEN"] = "your_token"
# os.environ["FIFTYONE_BRAIN_SIMILARITY_MOSAIC_CATALOG_NAME"] = "your_catalog"
# os.environ["FIFTYONE_BRAIN_SIMILARITY_MOSAIC_SCHEMA_NAME"] = "your_schema"
# os.environ["FIFTYONE_BRAIN_SIMILARITY_MOSAIC_ENDPOINT_NAME"] = "your_endpoint"

# These are critical for the SDK/MLflow auth
# os.environ["DATABRICKS_HOST"] = os.environ["FIFTYONE_BRAIN_SIMILARITY_MOSAIC_WORKSPACE_URL"]
# os.environ["DATABRICKS_TOKEN"] = os.environ["FIFTYONE_BRAIN_SIMILARITY_MOSAIC_PERSONAL_ACCESS_TOKEN"]

# For Twelve Labs
# os.environ["TL_API_KEY"]=your_Twelve_Labs_API_key

## 👌 Validate Authentication to Databricks + Twelve Labs
Make sure your token works by initializing a Databricks and Twelve Labs SDK client and confirming your identity.

In [ ]:
#Run this to check everything is in place:
from databricks.sdk import WorkspaceClient

client = WorkspaceClient()
me = client.current_user.me()
print("Authenticated as:", me.user_name)

In [ ]:
from twelvelabs import TwelveLabs
import os

tl_client = TwelveLabs(api_key=os.environ["TL_API_KEY"])

### 🏁 Create an endpoint in your catalog.

Be sure you have the following setup. Catalog -> Vector Search -> Endpoint. The FiftyOne integration will manage the rest. No worries about adding variables to your schema or settng up a vector search index, FiftyOne will manage it by you. 

In [ ]:
from databricks.vector_search.client import VectorSearchClient


# The following line automatically generates a PAT Token for authentication
client = VectorSearchClient()

# The following line uses the service principal token for authentication
# client = VectorSearchClient(service_principal_client_id=<CLIENT_ID>,service_principal_client_secret=<CLIENT_SECRET>)


client.create_endpoint(
    name="vector_search_fiftyone_twelve_labs_cluster",
    endpoint_type="STANDARD"
)


Wait until this endpoint is ready, any action before that can create a 500 or 400 HTTP Error.

## 📁 Load the Quickstart-Video Dataset and Launch FiftyOne
We will use the `quickstart-video` dataset from FiftyOne's built-in zoo to demonstrate embedding and vector indexing.

In [ ]:
import fiftyone as fo
import fiftyone.zoo as foz
import fiftyone.brain as fob

dataset = foz.load_zoo_dataset("quickstart-video")
session = fo.launch_app(dataset)

![quickstart-video](./assets/qsv.png)

## Generating Embeddings with Twelve Labs <img src="assets/twelve_labs_logo.jpg" alt="Image2" width="40"/>

Our next step is to add [Twelve Labs](https://www.twelvelabs.io/) embeddings to our video dataset. Twelve Labs provides state of the art embeddings, especially on videos that can be leveraged for powerful visualization and similarity worklows. 

Using our [plugin](https://github.com/danielgural/semantic_video_search) that we installed at the beginning, we can use our `create_twelve_labs_embeddings` operator to add these embeddings to our video dataset. To start, hit the " ` " button on your keyboard to open the operator list and type in  "create_twelve_labs_embeddings" and select the operator. Next, choose audio, visual, or both embeddings to generate on your video. You can also choose to compute on your whole dataset, selected samples, or your current view. 

If your dataset is large, you should check out [delegated operation](https://docs.voxel51.com/plugins/using_plugins.html#delegated-operations) for the plugin to avoid timing out the operator

![tl_plugin](./assets/tl_plugin.webp)

When we generate our Twelve Lab embeddings, Twelve Labs returns our embeddings in segments of clips. We store each of these embedding segments on our FiftyOne dataset as temporal detections to keep track of where each clip starts and end. 

Why do we need to do this? Well videos could be very long and when we do our search, we want to make sure we return the right **clip** not just the whole video! To start our search workflow, let's transform our dataset into a clip dataset using FiftyOne [`to_clips`](https://docs.voxel51.com/api/fiftyone.core.dataset.html?highlight=to_clips#fiftyone.core.dataset.Dataset.to_clips)

In [ ]:
import fiftyone.utils.video as fouv
import fiftyone.brain as fob

def create_clip_dataset(
    dataset: fo.Dataset,
    clip_field: str,
    new_dataset_name: str = "clips",
    overwrite: bool = True,
    viz: bool = False,
    sim: bool = False,
) -> fo.Dataset:
    clips = []
    clip_view = dataset.to_clips(clip_field)
    clip_dataset = fo.Dataset(name=new_dataset_name,overwrite=overwrite)
    i = 0
    last_file = ""
    samples = []
    for clip in clip_view:

        out_path = clip.filepath.split(".")[0] + f"_{i}.mp4"
        fpath = clip.filepath 
        fouv.extract_clip(fpath, output_path=out_path, support=clip.support)
        clip.filepath = out_path
        samples.append(clip)
        clip.filepath = fpath
        if clip.filepath == last_file:
            i += 1
        else:
            i = 0
        last_file = clip.filepath
    clip_dataset.add_samples(samples)
    clip_dataset.add_sample_field("Twelve Labs Marengo-retrieval-27 Embeddings", fo.VectorField)
    clip_dataset.set_field("Twelve Labs Marengo-retrieval-27 Embeddings", clip_view.values("Twelve Labs Marengo-retrieval-27.embedding"))
    
    return clip_dataset

In [ ]:
clip_dataset = create_clip_dataset(dataset, "Twelve Labs Marengo-retrieval-27", overwrite=True)

In [ ]:
# Need this to grab embeddings 
clip_view = dataset.to_clips( "Twelve Labs Marengo-retrieval-27")

## Query the Similarity Index

In [ ]:
# Might have to run twice if DB is not ready, make sure to update brain key
results = fob.compute_similarity(
            clip_dataset,
            brain_key="Twelve_Labs_Similarity",
            embeddings=clip_dataset.values("Twelve Labs Marengo-retrieval-27.embedding"),
            backend="mosaic",
            index_name="fiftyone_index",
        )

In [ ]:
# Query by first image sample
query =  clip_dataset.first().id
view = clip_dataset.sort_by_similarity(query, brain_key="Twelve_Labs_Similarity", k=3)
session.view = view

![img_sim](./assets/img_sim.png)

We can also search by text! In order to do so, we need to generate a text embedding from Twelve Labs and use that to search across our dataset. Let's try by searching for "fast food"

In [ ]:
# Query by text prompt
query_txt = "fast food"

res = tl_client.embed.create(
  model_name="Marengo-retrieval-2.7",
  text=query_txt,
)

embedding = res.text_embedding.segments[0].embeddings_float
view_txt = clip_dataset.sort_by_similarity(embedding, k=3, brain_key="Twelve_Labs_Similarity3")
session.view = view_txt

Sure enough, our first video we can see Burger King!

![txt_sim](./assets/txt_sim.png)

## Visualize Video Embeddings

We can also use our embeddings to visualize the distribution of our dataset! Using FiftyOne's `compute_visualization` we can generate our embedding map

In [ ]:
results = fob.compute_visualization(
            clip_dataset,
            method="umap", 
            brain_key="TwelveLabsVisualization",
            embeddings=clip_view.values("Twelve Labs Marengo-retrieval-27.embedding")
        )

session.dataset = clip_dataset

![emb_viz](./assets/embedding_viz.png)

## Cleanup (Optional)

In [ ]:
# Delete Mosaic index and run record
mosaic_index.cleanup()
dataset.delete_brain_run("mosaic_index")
#dataset.delete_brain_runs()